In [ ]:
# Example: Run a single experiment (for testing)
if False:  # Set to True to run a single test
    test_result = run_experiment(
        dataset="imdb",
        model="bert_base_qcfs",
        timestep=2,
        neuron_type="ParaInfNeuron_Text"
    )
    
    print("\nTest experiment completed!")

In [ ]:
# Experiment Configuration
EXPERIMENT_CONFIG = {
    "datasets": ["imdb", "ag_news"],
    "models": ["bert_base_qcfs"],
    "timesteps": [2, 4],
    "neuron_types": ["ParaInfNeuron_Text", "IFNeuron_Text"],
    "batch_size": 16,
    "learning_rate": 0.00001,
    "epochs": 5,
    "text_max_len": 256,
    "gpu_type": "A100"
}

# Text Classification Experiments with ANN-SNN Conversion

This notebook runs experiments for text classification using BERT and DistilBERT models with different spiking neuron types.

In [ ]:
import os
import subprocess
import pandas as pd
import json
import time
from itertools import product
from datetime import datetime

# Experiment Configuration
EXPERIMENT_CONFIG = {
    "datasets": ["imdb", "ag_news"],
    "models": ["bert_base_qcfs"],
    "timesteps": [2, 4],
    "neuron_types": ["ParaInfNeuron_Text", "IFNeuron_Text"],
    "batch_size": 16,
    "learning_rate": 0.00001,
    "epochs": 5,
    "text_max_len": 256,
    "gpu_type": "A100"
}

# Create results directory
os.makedirs("./text_results", exist_ok=True)
os.makedirs("./checkpoints", exist_ok=True)

print("Experiment Configuration:")
for key, value in EXPERIMENT_CONFIG.items():
    print(f"{key}: {value}")

In [ ]:
def run_experiment(dataset, model, timestep, neuron_type):
    """Run a single experiment with the given parameters"""
    
    # Prepare the command
    cmd = f"python Parallel_Conversion/main.py \\
        --dataset TextCLS \\
        --net_arch {model} \\
        --savedir ./checkpoints \\
        --neuron_type {neuron_type} \\
        --text_dataset {dataset} \\
        --text_max_len {EXPERIMENT_CONFIG['text_max_len']} \\
        --time_step {timestep} \\
        --trainsnn_epochs {EXPERIMENT_CONFIG['epochs']} \\
        --batchsize {EXPERIMENT_CONFIG['batch_size']} \\
        --lr {EXPERIMENT_CONFIG['learning_rate']} \\
        --measure_efficiency \\
        --gpu_type {EXPERIMENT_CONFIG['gpu_type']} \\
        --dev 0"
    
    print(f"\n{'='*50}")
    print(f"Running experiment:")
    print(f"  Dataset: {dataset}")
    print(f"  Model: {model}")
    print(f"  Timestep: {timestep}")
    print(f"  Neuron Type: {neuron_type}")
    print(f"{'='*50}")
    
    # Run the experiment
    start_time = time.time()
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=3600)
        
        # Save results
        experiment_name = f"{dataset}_{model}_{neuron_type}_t{timestep}"
        result_file = f"./text_results/{experiment_name}_result.txt"
        
        with open(result_file, 'w') as f:
            f.write(f"Experiment: {experiment_name}\n")
            f.write(f"Command: {cmd}\n")
            f.write(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"\n=== STDOUT ===\n")
            f.write(result.stdout)
            if result.stderr:
                f.write(f"\n=== STDERR ===\n")
                f.write(result.stderr)
            f.write(f"\nReturn Code: {result.returncode}\n")
            f.write(f"Execution Time: {time.time() - start_time:.2f} seconds\n")
        
        print(f"Experiment completed. Results saved to: {result_file}")
        print(f"Return code: {result.returncode}")
        print(f"Execution time: {time.time() - start_time:.2f} seconds")
        
        return {
            'dataset': dataset,
            'model': model,
            'timestep': timestep,
            'neuron_type': neuron_type,
            'return_code': result.returncode,
            'execution_time': time.time() - start_time,
            'result_file': result_file,
            'stdout': result.stdout,
            'stderr': result.stderr
        }
        
    except subprocess.TimeoutExpired:
        print(f"Experiment timed out after 3600 seconds")
        return {
            'dataset': dataset,
            'model': model,
            'timestep': timestep,
            'neuron_type': neuron_type,
            'return_code': -1,
            'execution_time': time.time() - start_time,
            'error': 'Timeout'
        }
    except Exception as e:
        print(f"Experiment failed with error: {e}")
        return {
            'dataset': dataset,
            'model': model,
            'timestep': timestep,
            'neuron_type': neuron_type,
            'return_code': -1,
            'execution_time': time.time() - start_time,
            'error': str(e)
        }